In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go
from ipywidgets import Output

coverage_data  = pd.read_excel('results_df.xlsx')
geo_df = gpd.read_file('capital_region_municipalities.geojson')
house_prices = pd.read_excel('house_price.xlsx')
house_prices.set_index('Municipality', inplace=True)

# Initialize weights and sliders
facilities = ['company', 'eat', 'gym', 'hospital', 'park', 'school', 'shop', 'station']
weights = {fac: 0 for fac in facilities}
sliders = {fac: widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description=f'{fac.capitalize()}:') for fac in weights}
total_weight_label = widgets.Label('Total Weight: 0.00')
button = widgets.Button(description="Start Update", disabled=True)

# Create output containers
chart_container = widgets.Output()
map_container = widgets.Output()

# Function to update total weight and check constraints
def update_total_weight(change):
    total_weight = sum(slider.value for slider in sliders.values())
    total_weight_label.value = f"Total Weight: {total_weight:.2f}"
    button.disabled = not np.isclose(total_weight, 1.0)

# Connect sliders to the weight updating function
for slider in sliders.values():
    slider.observe(update_total_weight, 'value')

def on_button_clicked(b):
    print("Button clicked")  # Debug: Check if this gets printed when you click the button
    update_visuals()

def update_visuals():
    print("Updating visuals...")  # Debug: Check if entering the function
    try:
        merged_df = geo_df.merge(coverage_data, left_on='name_gn_spell_spellofna_text', right_on='name_gn_spell_spellofna_text', how='left')
        for fac, slider in sliders.items():
            if fac + '_coverage' in merged_df.columns:  # Ensure column exists
                merged_df[fac + '_score'] = merged_df[fac + '_coverage'] * slider.value
                print(f"Scores for {fac} updated.")  # Confirm score updates

        merged_df['total_score'] = merged_df[[f"{fac}_score" for fac in facilities if fac + '_coverage' in merged_df.columns]].sum(axis=1)
        print("Total scores calculated.")  # Check total scores

        create_map(merged_df)
        top_municipalities = merged_df.nsmallest(3, 'total_score')['name_gn_spell_spellofna_text'].tolist()
        create_chart(top_municipalities)
    except Exception as e:
        print("Error in update_visuals:", e)  # Print any errors to debug

button.on_click(on_button_clicked)  # Ensure this is correctly hooked up


def create_map(merged_df):
    with map_container:
        clear_output(wait=True)  # Ensure clearing the output
        print("Creating map...")  # Debug message
        m = folium.Map(location=[55.6761, 12.5683], zoom_start=10)
        folium.Choropleth(
            geo_data=merged_df.to_json(),
            data=merged_df,
            columns=['name_gn_spell_spellofna_text', 'total_score'],
            key_on='feature.properties.name_gn_spell_spellofna_text',
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Score'
        ).add_to(m)
        display(m)  # Make sure to call display within the `with` block

def create_chart(municipalities):
    with chart_container:
        clear_output(wait=True)  # Ensure clearing the output
        print("Creating chart...")  # Debug message
        traces = []
        for municipality in municipalities:
            trace = go.Scatter(
                x=house_prices.columns,
                y=house_prices.loc[municipality],
                mode='lines+markers',
                name=municipality
            )
            traces.append(trace)
        fig = go.Figure(data=traces)
        fig.update_layout(title='Price Trends', xaxis_title='Year', yaxis_title='Price')
        display(fig)  # Make sure to call display within the `with` block


# Layout the widgets and output
control_box = widgets.VBox([total_weight_label] + list(sliders.values()) + [button])
layout = widgets.HBox([control_box, widgets.VBox([map_container, chart_container])])
display(layout)

